In [1]:
!pip install langchain langchain-text-splitters langchain-community bs4


   ------------- -------------------------- 1/3 [beautifulsoup4]
   ---------------------------------------- 3/3 [bs4]



# Preparation

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [8]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)


# 1. Indexing

### Load Documents from an URl

In [ ]:
# bs4：BeautifulSoup，用于解析 HTML
# WebBaseLoader：LangChain 提供的网页加载器，用来抓取网页并转成文档对象
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
headers = {
    "User-Agent": "SpagBolOntology/0.1 (charlie; GuanSc@proton.com)"
}

# SoupStrainer 的作用：
# 👉 只解析 HTML 中符合条件的部分
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

# 告诉 loader：
# 👉 用 BeautifulSoup
# 👉 且只解析 bs4_strainer 指定的内容
# web_paths 是 tuple（即使只有一个 URL 也要加逗号）
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
    header_template={
        "User-Agent": "Rag-test/0.1 (charlie; ucabs95@ucl.ac.uk)"
    },
)

docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 43047


In [3]:
print(type(docs))

<class 'list'>


### Splitting documents

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


### Storing documents

In [9]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['1f40c92a-de4d-45d9-bc18-cd43d35f182a', 'a8cebb83-73f5-4058-a1ff-d3295590395d', '29a94d89-0b54-4276-9430-bcedf17e7fe8']


# 2. Retrieval and Generation

### RAG agents

response_format="content_and_artifact" 是什么？

表示这个 tool 返回两个部分：
| 返回部分       | 含义              |
| ---------- | --------------- |
| `content`  | 给 LLM 用来“读”的文本  |
| `artifact` | 给系统/程序继续处理的原始对象 |


### Build a tool

In [12]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        # 每个文档被格式化为：
        # Source: {元数据}
        # Content: {正文}
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    # | 返回值              | 用途              |
    # | ---------------- | --------------- |
    # | `serialized`     | 👉 给 LLM 当“上下文” |
    # | `retrieved_docs` | 👉 给程序后处理       |

    return serialized, retrieved_docs

### Assemble an agent

In [14]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.5
)

In [15]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

### 用 LangGraph/LangChain 的“状态机式 Agent”做 流式执行。

### 1）agent.stream(...) 本质是什么？

agent.stream(...) **不是一次性算完给你一个结果**，而是返回一个“可迭代对象 / 生成器（generator）”。

- 你可以把 agent 想成一个会经历多步的流程：
  “读输入 → 推理 →（可能）调用工具 → 得到工具结果 → 再推理 → 输出回答”

- **每走一步，它就 yield 一个 event 给你**

- 所以你可以在 for event in ... 里实时看到它的进展

### 2）for event in ... 为什么能工作？

因为 agent.stream(...) 返回的东西类似于：
```python
def stream(...):
    yield event1
    yield event2
    yield event3
    ...
```

于是 Python 的 for 循环就会：

- 拿到 event1

- 执行循环体（你后面打印 message）

- 再拿 event2

……直到 stream 结束

这就是 “streaming” 的含义：**不是等最后一步，而是边做边吐出中间状态。**

### 3）传入的第一个参数：{"messages": [...]} 到底是什么？
```python
{"messages": [{"role": "user", "content": query}]}
```

这是给 agent 的初始状态（initial state）。

#### 3.1 为什么叫 messages？

在 LangChain / LangGraph 里，很多 agent 都用一个“状态字典 state”来驱动执行，最核心的字段通常就是：

- messages: 目前对话里所有消息的列表（聊天历史）

- agent 的每一步，会基于 messages 继续追加新的消息（AI 回复、工具返回等）

#### 3.2 messages 为什么是 list？

因为对话是多轮的，必须能表示：

- 第 1 轮 user

- 第 1 轮 assistant

-第 2 轮 user

- tool message

…

所以用列表按时间顺序存。

#### 3.3 {"role": "user", "content": query} 是什么格式？

这就是典型的 ChatML / OpenAI chat 格式的简化版：

- role="user"：这条消息来自用户

- content=query：消息内容就是你上面拼出来的那段字符串

换句话说，这一坨的含义就是：

**“把用户的提问 query 作为对话的第一条消息，交给 agent。”**

### 4）stream_mode="values" 到底控制了什么？

stream_mode 决定了 stream 过程中 yield 出来的 event 长什么样。

你设成：
```python
stream_mode="values"
```

一般表示：

每次 yield 的 event 都给你“当前完整 state 的值”。

也就是 event 往往像这样（概念示意）：

```python
event = {
  "messages": [
    {"role": "user", "content": "..."},
    {"role": "assistant", "content": "..."},
    {"role": "tool", "content": "..."},
    ...
  ],
  # 可能还有别的 state 字段
}
```
#### 4.1 为什么这很有用？

因为你可以随时从 event["messages"] 看到目前为止整个对话状态（包含新追加的东西）。

#### 4.2 如果不是 values 会怎样？

不同框架实现略有差异，但概念上：

- values：给“全量状态”

- updates：只给“这一步新变化的那部分”（增量）

- debug：给更多内部执行信息（节点名/路由/耗时等）

用 values 的好处是：最直观、最不容易漏信息，代价是 event 可能更大。

### 5）这段代码在执行时会经历什么“步骤”？

假设 agent 是一个会用搜索工具的 agent，它可能产生下面这些 event（示意）：

- 第一次 yield：刚把 user message 放进去

 event["messages"] = [HumanMessage(query)]

- 第二次 yield：assistant 先给一个中间想法/计划（或直接开始回答）

 event["messages"] 末尾多了 AIMessage(...)

- 第三次 yield：assistant 决定调用工具（比如 search）

 末尾多了一个 ToolCall 或“要调用工具”的消息

- 第四次 yield：工具返回结果

 末尾多了 ToolMessage(...)
 
- 第五次 yield：assistant 综合工具结果，输出最终回答

 末尾多了最终 AIMessage(...)

你之所以用 stream，就是为了能看到这些中间阶段。

In [ ]:
# 在 LLM 里：
# \n\n 会被模型强烈感知为 “任务切换 / 新阶段”
# 比单纯用句号更利于触发 step-by-step 行为
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    # 初始状态
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    
    # event["messages"]
    # 这是 当前对话状态中的所有消息，
    # [-1]：当前这一步 agent 最新产生的消息

    # .pretty_print() 是什么？
    # 这是 LangChain / LangGraph Message 对象自带的方法，用来：
    # 自动区分角色（Human / AI / Tool）
    # 美化输出
    # 在终端里清晰显示
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_9X3chempPJwwuNs0hVeKMqe0)
 Call ID: call_9X3chempPJwwuNs0hVeKMqe0
  Args:
    query: standard method for Task Decomposition
  retrieve_context (call_IGVOpmzJplhII07BhKKyACXZ)
 Call ID: call_IGVOpmzJplhII07BhKKyACXZ
  Args:
    query: common extensions of the standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructio

# RAG chains

Another common approach is a two-step chain, in which we always run a search (potentially using the raw user query) and incorporate the result as context for a single LLM query.<br> 

This results in a single inference call per query, buying reduced latency at the expense of flexibility.<br>

In this approach we no longer call the model in a loop, but instead make a single pass.<br>

We can implement this chain by removing tools from the agent and instead incorporating the retrieval step into a custom prompt:<br>

In [ ]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest
# ModelRequest 可以理解为：
# “一次即将发送给 LLM 的完整请求对象”
# 里面通常包含：
#  request.state：Agent 当前的状态（messages、memory 等）
#  request.messages：最终要送进模型的消息
# 其他模型调用相关信息

# “在模型每次被调用前，运行这个函数，
# 并把它的返回值当作 system prompt 注入进去。”
@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    
    # 把 RAG 内容注入为 system message，而不是 user message
    # 这意味着：
    # 模型会把它当作“最高优先级指令”
    # 用户无法轻易 override
    # 更稳定、可控
    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message
# 最后会把 system prompt + 对话 messages 一起送给模型


agent = create_agent(model, tools=[], middleware=[prompt_with_context])

In [18]:
query = "What is task decomposition?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is task decomposition?
================================== Ai Message ==================================

Task decomposition is the process of breaking down a complex task or problem into smaller, more manageable sub-tasks or steps. This makes it easier to understand, plan, and execute the overall task by tackling each smaller part one at a time.

Task decomposition can be done in several ways:

1. Using Large Language Models (LLMs) with simple prompts, such as asking "What are the steps for achieving XYZ?" or "What are the subgoals for completing XYZ?"
2. Employing task-specific instructions tailored to the domain, for example, instructing an LLM to "Write a story outline" when working on a novel.
3. Incorporating human input to guide the breakdown based on expertise or experience.

Additionally, there are more advanced approaches like combining LLMs with external classical planners (LLM+P). Here, th

The above RAG chain incorporates retrieved context into a single system message for that run.<br>
As in the agentic RAG formulation, we sometimes want to **include raw source documents in the application state** to have access to **document metadata**. We can do this for the two-step chain case by:
1. Adding a key to the state to store the retrieved documents
2. Adding a new node via a pre-model hook to populate that key (as well as inject the context).

### 1）State：给 agent 的状态加一个字段 context
```python
class State(AgentState):
    context: list[Document]
```

- AgentState 是框架里约定的“状态字典”基类，通常至少包含 messages。

- 你这里扩展了一个字段：context，类型是 list[Document]。

- Document 是 LangChain 的文档对象，常见属性：

  -  page_content：正文文本

  - metadata：来源、页码、URL 等

**目的：** 让 agent 在运行过程中能把“检索到的文档”作为结构化数据存下来，而不仅仅是拼到 prompt 里。

### 2）RetrieveDocumentsMiddleware：一个“模型调用前”的中间件
```python
class RetrieveDocumentsMiddleware(AgentMiddleware[State]):
    state_schema = State
```

AgentMiddleware[State] 表示这个中间件操作的 state 遵循你定义的 State 结构（带 context）。

state_schema = State 是告诉框架：这个 middleware 读写 state 时按 State 校验/推断字段。

### 3）before_model：关键钩子，在每次 LLM 调用前运行
```python
def before_model(self, state: AgentState) -> dict[str, Any] | None:
```

语义是：

每当 agent 准备调用模型（LLM）之前，先把当前 state 交给你；
你可以返回一个 dict 来“覆盖/追加”本次模型调用用到的输入（以及更新 state），也可以返回 None 表示不改。

注意：不同版本实现细节略有不同，但这个钩子的目标就是 **“改输入、改 state”。**

In [19]:
from typing import Any
from langchain_core.documents import Document
from langchain.agents.middleware import AgentMiddleware, AgentState


class State(AgentState):
    context: list[Document]


class RetrieveDocumentsMiddleware(AgentMiddleware[State]):
    state_schema = State

    def before_model(self, state: AgentState) -> dict[str, Any] | None:
        last_message = state["messages"][-1]
        retrieved_docs = vector_store.similarity_search(last_message.text)

        docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

        augmented_message_content = (
            f"{last_message.text}\n\n"
            "Use the following context to answer the query:\n"
            f"{docs_content}"
        )
        return {
            "messages": [last_message.model_copy(update={"content": augmented_message_content})],
            "context": retrieved_docs,
        }


agent = create_agent(
    model,
    tools=[],
    middleware=[RetrieveDocumentsMiddleware()],
)